In [1]:
setwd('/import/hd//sc')
library(Seurat)

Warning message:
"package 'Seurat' was built under R version 4.2.3"
The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject



In [2]:
sce = readRDS('./scRNA.rds')

In [3]:
table(sce$celltype)


     ADs     Endo      Fib Lymphoid    Mural  Myeloid 
    4025    12465    44889     5107     6472    15914 

In [4]:
table(sce$orig.ident)


        OA_1         OA_2         OA_3    healthy_1    healthy_2    healthy_3 
        9169        10914         8697        11700        11449        10005 
     sn_OA_1      sn_OA_2 sn_healthy_1 
       10034         7487         9417 

In [8]:
dim(as.matrix(sce@assays$RNA@counts))
Idents(sce) = 'orig.ident'
immune.combined = subset(sce,idents = c( 'sn_OA_1',  'sn_OA_2' ,'sn_healthy_1' ),invert =TRUE)
mca2=CreateSeuratObject(immune.combined@assays$RNA@data,assay = "RNA",min.cells = 10,meta.data = immune.combined@meta.data)
immune.combined = mca2
immune.combined

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 40.5 GiB"


[1] 61134 88872

An object of class Seurat 
23545 features across 61934 samples within 1 assay 
Active assay: RNA (23545 features, 0 variable features)

In [9]:
sce = immune.combined

In [10]:
Idents(sce) = 'celltype'

In [11]:
sce1 = subset(sce,idents = c('Lymphoid','ADs','Endo'))

In [12]:
meta1  = sce1@meta.data[,c(1,12)]

In [13]:
fib = readRDS('./scRNA.fibro.rds')

In [56]:
table(fib$celltype)


 Fib0  Fib1  Fib2  Fib3 
20992 12476 10875   546 

In [14]:
colnames(fib@meta.data)

[1] "orig.ident"       "nCount_RNA"       "nFeature_RNA"     "percent.mt"      
 [5] "RNA_snn_res.0.8"  "seurat_clusters"  "RNA_snn_res.0.2"  "group"           
 [9] "RNA_snn_res.0.1"  "RNA_snn_res.0.05" "RNA_snn_res.0.01" "celltype"

In [15]:
fib.meta = fib@meta.data[,c(1,12)]

In [16]:
mye = readRDS('./scRNA.macro.rds')

In [17]:
mye.meta = mye@meta.data[,c(1,12)]

In [18]:
mural = readRDS('/import/a.IMCDataAnalysi2/annotation/scRNA.mural_reClust.seurat.rds')

In [19]:
mural.meta = mural@meta.data[,c(1,13)]

In [20]:
colnames(mural.meta)[2] = 'celltype'

In [21]:
merged = rbind(fib.meta,mye.meta,mural.meta,meta1)

In [22]:
sce = AddMetaData(sce,metadata = merged)

In [23]:
Idents(sce) = 'celltype'

In [27]:
sce2 = subset(sce,idents = na.omit(unique(sce$celltype)))

In [33]:
sce2

An object of class Seurat 
23545 features across 61850 samples within 1 assay 
Active assay: RNA (23545 features, 0 variable features)

In [55]:
table(sce2$celltype)


          ADs           ECs          Endo          Fib0          Fib1 
           39           642          8181         20971         12432 
         Fib2 Inflamm.Macro      Lymphoid      Pericyte   Resid.Macro 
            2          1469          3964          2143          3012 
          SMC    SPP1_macro        macro1        macro2        macro3 
         3251           372           989           202           197 
       macro4          mono 
           63          3921 

In [57]:
Idents(sce2) = 'celltype'

In [ ]:
Idents(sce2) = 'celltype'

sce.markers <- FindAllMarkers(object = sce2, only.pos = TRUE, 
                              min.pct = 0.25, 
                              thresh.use = 0.25)
 

library(dplyr) 
# 不同seurat版本的 avg_logFC 不一样 
top5 <- sce.markers %>% group_by(cluster) %>% top_n(5, avg_log2FC)

Calculating cluster Fib0

Calculating cluster Endo



In [ ]:
library(tidyr)

In [ ]:
library(reshape2)
a = sce.markers %>% pivot_wider( names_from = cluster, values_from = gene)

In [50]:
list1 <- as.list(a[,6:dim(a)[2]])

In [51]:

my_list_no_na <- lapply(list1, unique)
save(my_list_no_na,file = 'my_list_no_na2.rda')

In [ ]:
saveRDS(sce2,file = 'sce2.merged.rds')

In [42]:
getwd()

[1] "/import/hd/sc"

In [43]:
table(sce2$celltype)


          ADs           ECs          Endo          Fib0          Fib1 
           39           642          8181         20971         12432 
         Fib2 Inflamm.Macro      Lymphoid      Pericyte   Resid.Macro 
            2          1469          3964          2143          3012 
          SMC    SPP1_macro        macro1        macro2        macro3 
         3251           372           989           202           197 
       macro4          mono 
           63          3921 

In [52]:
length(my_list_no_na)

[1] 16

In [53]:
names(my_list_no_na)

[1] "Fib0"          "Endo"          "Resid.Macro"   "Fib1"         
 [5] "mono"          "Lymphoid"      "macro1"        "SPP1_macro"   
 [9] "SMC"           "macro3"        "Inflamm.Macro" "macro4"       
[13] "Pericyte"      "macro2"        "ECs"           "ADs"